In [1]:
# Import packages
import pandas as pd
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load and preprocess data

In [2]:
# Load datasets
df = pd.read_excel('Dokumenter\ChildrenWristFeaturesV2_TSF_new.xlsx')
df.fillna(df.mean(), inplace=True)

In [19]:
# Initialize features and targets/labels
X = df.iloc[:, 1:32]
Y = df.iloc[:, 32]

# Splits datasets into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Binarize the output
#Y = label_binarize(Y, classes=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12])
#n_classes = Y.shape[1]

# Data preprocessing, scaling datasets/standardization
scal_data = StandardScaler()
X = scal_data.fit_transform(X)

from sklearn.preprocessing import MinMaxScaler
# Data preprocessing, normalize datasets/normalization
#min_max = MinMaxScaler()
#X = min_max.fit_transform(X)

In [19]:
#import keras
from keras.models import Sequential
from keras.layers import Dense
# Define the keras model
keras_model = Sequential()
#keras_model.add(Flatten())
keras_model.add(Dense(12, input_dim=31, activation='relu')) # input dim
keras_model.add(Dense(8, activation='relu'))
keras_model.add(Dense(11, activation='softmax'))  #activation=’softmax’ for classification
# Compile the keras model
keras_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# New train, test, split
#X_train_new_km, X_test_new_km, y_train_new_km, y_test_new_km = train_test_split(X, Y, test_size=0.2, random_state=42)
# Fit the keras model on the dataset
keras_model.fit(X_train, y_train, epochs=5, batch_size=10) #verbose=0, epochs=150
# Evaluate the keras model (training set)
_, accuracy_train = keras_model.evaluate(X_train, y_train) #verbose=0
print('Train Accuracy: %.2f' % (accuracy_train*100))
# Evaluate the keras model (test set)
_, accuracy_test = keras_model.evaluate(X_test, y_test) #verbose=0
print('Test Accuracy: %.2f' % (accuracy_test*100))

Epoch 1/5


ValueError: in user code:

    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Anaconda\envs\Classification\lib\site-packages\tensorflow\python\k

In [11]:
keras_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_6 (Dense)              (None, 11)                99        
Total params: 311
Trainable params: 311
Non-trainable params: 0
_________________________________________________________________


In [14]:
X.shape

(136653, 31)

# Multi-class classification with Keras

## First trial - overview

In [24]:
%%time
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(1024, input_dim=31, activation='relu'))  # input layer, nodes=32
#	model.add(Dense(32, activation='relu'))                # hidden layer, nodes=32
#	model.add(Dense(16, activation='relu'))                  # hidden layer, nodes=16
	model.add(Dense(11, activation='softmax'))             # output layer
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=5, verbose=1)  #epoch=200
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) #Baseline change to Accuracy

2733/2733 [==============================] - 2s 705us/step - loss: 1.3087 - accuracy: 0.4616
Accuracy: 46.82% (0.48%)
Wall time: 4min 59s


## Second trial - one hidden layer with many neurons

In [9]:
%%time
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, activation='relu'))  # input layer, nodes=32
	model.add(Dense(5000, activation='relu'))                # hidden layer, nodes=32
#	model.add(Dense(16, activation='relu'))                  # hidden layer, nodes=16
	model.add(Dense(11, activation='softmax'))             # output layer
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=10, verbose=1)  #epoch=200, batch_size=1
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) #Baseline change to Accuracy

Epoch 1/50
12299/12299 [==============================] - 25s 2ms/step - loss: 1.4767 - accuracy: 0.4444
Epoch 2/50
12299/12299 [==============================] - 26s 2ms/step - loss: 1.3423 - accuracy: 0.4618
Epoch 3/50
12299/12299 [==============================] - 24s 2ms/step - loss: 1.3081 - accuracy: 0.4688
Epoch 4/50
12299/12299 [==============================] - 24s 2ms/step - loss: 1.2895 - accuracy: 0.4747
Epoch 5/50
12299/12299 [==============================] - 24s 2ms/step - loss: 1.2795 - accuracy: 0.4749
Epoch 6/50
12299/12299 [==============================] - 26s 2ms/step - loss: 1.2698 - accuracy: 0.4769
Epoch 7/50
12299/12299 [==============================] - 25s 2ms/step - loss: 1.2619 - accuracy: 0.4797
Epoch 8/50
12299/12299 [==============================] - 24s 2ms/step - loss: 1.2549 - accuracy: 0.4812
Epoch 9/50
12299/12299 [==============================] - 27s 2ms/step - loss: 1.2516 - accuracy: 0.4825
Epoch 10/50
12299/12299 [==============================

## Third trial - deep network (many hidden layers with few neurons in each layer)

In [10]:
%%time
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, activation='relu'))    # input layer, nodes=32
	model.add(Dense(100, activation='relu'))                 # hidden layer, nodes=32
	model.add(Dense(200, activation='relu'))                 # hidden layer, nodes=16
	model.add(Dense(100, activation='relu'))
#	model.add(Dense(3, activation='relu'))
#	model.add(Dense(3, activation='relu'))
#	model.add(Dense(3, activation='relu'))
#	model.add(Dense(3, activation='relu'))
	model.add(Dense(11, activation='softmax'))             # output layer
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=10, verbose=1)  #epoch=200, batch_size=1
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(deep, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) #Baseline change to Accuracy

Epoch 1/50
12299/12299 [==============================] - 14s 1ms/step - loss: 1.9752 - accuracy: 0.4409
Epoch 2/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9599 - accuracy: 0.4411
Epoch 3/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9599 - accuracy: 0.4411
Epoch 4/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9599 - accuracy: 0.4411
Epoch 5/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9598 - accuracy: 0.4411
Epoch 6/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9599 - accuracy: 0.4411
Epoch 7/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9598 - accuracy: 0.4411
Epoch 8/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9598 - accuracy: 0.4411
Epoch 9/50
12299/12299 [==============================] - 13s 1ms/step - loss: 1.9598 - accuracy: 0.4411
Epoch 10/50
12299/12299 [==============================

## Fourth trial - single hidden layer with many units/neurons

In [22]:
%%time
# Multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# Convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# Define baseline model
def baseline_model():
	# Create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, activation='relu'))  # input layer, nodes=32
	model.add(Dense(5000, activation='relu'))                # hidden layer, nodes=32
#	model.add(Dense(16, activation='relu'))                  # hidden layer, nodes=16
	model.add(Dense(11, activation='softmax'))             # output layer
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=10, verbose=1)  #(epoch=200)/epoch=50, batch_size=1
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) #Baseline change to Accuracy

Epoch 1/50
12299/12299 [==============================] - 24s 2ms/step - loss: 1.2419 - accuracy: 0.4916
Epoch 2/50
12299/12299 [==============================] - 23s 2ms/step - loss: 1.0673 - accuracy: 0.5489
Epoch 3/50
12299/12299 [==============================] - 25s 2ms/step - loss: 0.9797 - accuracy: 0.5843
Epoch 4/50
12299/12299 [==============================] - 24s 2ms/step - loss: 0.9211 - accuracy: 0.6094
Epoch 5/50
12299/12299 [==============================] - 24s 2ms/step - loss: 0.8809 - accuracy: 0.6243
Epoch 6/50
12299/12299 [==============================] - 23s 2ms/step - loss: 0.8509 - accuracy: 0.6389
Epoch 7/50
12299/12299 [==============================] - 24s 2ms/step - loss: 0.8274 - accuracy: 0.6490
Epoch 8/50
12299/12299 [==============================] - 25s 2ms/step - loss: 0.8074 - accuracy: 0.6571
Epoch 9/50
12299/12299 [==============================] - 24s 2ms/step - loss: 0.7898 - accuracy: 0.6661
Epoch 10/50
12299/12299 [==============================

In [ ]:
history_callback = model.fit(X_train, y_train, batch_size=10, epochs=100, verbose=1)
pd.DataFrame(history_callback.history).plot()
eval_tr = model.evaluate(X_train, y_train)
eval_tr
eval_ts = model.evaluate(X_test, y_test)
eval_ts


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def make_model(optimizer="adam", hidden_size=32):
    model = Sequential([
        Dense(4, input_shape=(X_train.shape[1],), name='dense_1'),
        Activation('relu', name='activation_1'),
        Dense(4, name='dense_2'),
        Activation('relu', name='activation_2'),
        Dense(1, name='dense_3'),
        Activation('sigmoid', name='activation_3')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

clf = KerasClassifier(make_model)
param_grid = {'epochs': [10, 50, 100],  
              'hidden_size': [(4,4), (4,16), (16,16)]}
grid = GridSearchCV(clf, param_grid=param_grid, cv=5, return_train_score=True)


# Hyperparameter tuning with keras model

## Tuning on batch size and epochs parameter

In [6]:
%%time
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# Define baseline model
def keras_model():      
	# Create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, activation='relu'))           #kernel_initializer=init, (units=10) previous result                       
	model.add(Dense(11, activation='softmax'))                          #kernel_initializer=init
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# Create model
model = KerasClassifier(build_fn=keras_model, verbose=1)  
# Define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)   
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)    # cv=3, (n_jobs=-1) <--- BAD IDEA
grid_result = grid.fit(X, Y)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
9111/9111 [==============================] - 8s 861us/step - loss: 1.8209 - accuracy: 0.4214
Epoch 2/10
9111/9111 [==============================] - 7s 811us/step - loss: 1.4453 - accuracy: 0.4516
Epoch 3/10
9111/9111 [==============================] - 7s 799us/step - loss: 1.4022 - accuracy: 0.4536
Epoch 4/10
9111/9111 [==============================] - 7s 811us/step - loss: 1.3870 - accuracy: 0.4552
Epoch 5/10
9111/9111 [==============================] - 7s 804us/step - loss: 1.3737 - accuracy: 0.4574
Epoch 6/10
9111/9111 [==============================] - 7s 800us/step - loss: 1.3612 - accuracy: 0.4618
Epoch 7/10
9111/9111 [==============================] - 7s 794us/step - loss: 1.3506 - accuracy: 0.4646
Epoch 8/10
9111/9111 [==============================] - 7s 806us/step - loss: 1.3405 - accuracy: 0.4671
Epoch 9/10
9111/9111 [==============================] - 8s 866us/step - loss: 1.3348 - accuracy: 0.4695
Epoch 10/10
4556/4556 [==============================] - 3s 658u

## Tuning on weight initialization parameter

### First trial

In [7]:
%%time
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# Define baseline model
def keras_model(init_mode='uniform'):      
	# Create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, kernel_initializer=init_mode, activation='relu'))   #kernel_initializer=init, (units=10) previous result                       
	model.add(Dense(11, kernel_initializer=init_mode, activation='softmax'))              #kernel_initializer=init
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# Create model
model = KerasClassifier(build_fn=keras_model, epochs=50, batch_size=10, verbose=1)  
# Define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)   
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)    # cv=3, (n_jobs=-1) <--- BAD IDEA
grid_result = grid.fit(X, Y)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
9111/9111 [==============================] - 8s 900us/step - loss: 1.4604 - accuracy: 0.4611
Epoch 2/50
9111/9111 [==============================] - 9s 942us/step - loss: 1.3540 - accuracy: 0.4690
Epoch 3/50
9111/9111 [==============================] - 8s 892us/step - loss: 1.3346 - accuracy: 0.4723
Epoch 4/50
9111/9111 [==============================] - 9s 936us/step - loss: 1.3224 - accuracy: 0.4760
Epoch 5/50
9111/9111 [==============================] - 10s 1ms/step - loss: 1.3115 - accuracy: 0.4786
Epoch 6/50
9111/9111 [==============================] - 9s 1ms/step - loss: 1.3025 - accuracy: 0.4798
Epoch 7/50
9111/9111 [==============================] - 9s 1ms/step - loss: 1.2925 - accuracy: 0.4825
Epoch 8/50
9111/9111 [==============================] - 8s 868us/step - loss: 1.2836 - accuracy: 0.4872
Epoch 9/50
9111/9111 [==============================] - 7s 816us/step - loss: 1.2749 - accuracy: 0.4901
Epoch 10/50
9111/9111 [==============================] - 7s 813us/ste

# Tuning on the activation function parameter

## First trial

In [8]:
%%time
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# Define baseline model
def keras_model(activation='relu'):      
	# Create model
	model = Sequential()
	model.add(Dense(10, input_dim=31, kernel_initializer='normal', activation=activation))   #kernel_initializer=init, (units=10) previous result                       
	model.add(Dense(11, kernel_initializer='normal', activation='softmax'))              #kernel_initializer=init
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# Create model
model = KerasClassifier(build_fn=keras_model, epochs=50, batch_size=10, verbose=1)  
# Define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)   
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)    # cv=3, (n_jobs=-1) <--- BAD IDEA
grid_result = grid.fit(X, Y)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
9111/9111 [==============================] - 9s 993us/step - loss: 1.9530 - accuracy: 0.4477
Epoch 2/50
9111/9111 [==============================] - 9s 1ms/step - loss: 1.9364 - accuracy: 0.4487
Epoch 3/50
9111/9111 [==============================] - 8s 874us/step - loss: 1.9364 - accuracy: 0.4487
Epoch 4/50
9111/9111 [==============================] - 8s 889us/step - loss: 1.9364 - accuracy: 0.4487
Epoch 5/50
9111/9111 [==============================] - 8s 873us/step - loss: 1.9364 - accuracy: 0.4487
Epoch 6/50
9111/9111 [==============================] - 8s 876us/step - loss: 1.9365 - accuracy: 0.4487
Epoch 7/50
9111/9111 [==============================] - 8s 867us/step - loss: 1.9364 - accuracy: 0.4487
Epoch 8/50
9111/9111 [==============================] - 8s 906us/step - loss: 1.9364 - accuracy: 0.4487
Epoch 9/50
9111/9111 [==============================] - 8s 887us/step - loss: 1.9365 - accuracy: 0.4487
Epoch 10/50
9111/9111 [==============================] - 8s 879us/

## Tuning on units/neurons parameter

### First trial

In [31]:
%%time
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# define baseline model
def keras_model(neurons=1):                  #optimizer='rmsprop', init='glorot_uniform'
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=31, activation='relu'))    #kernel_initializer=init
	model.add(Dense(11, activation='softmax'))        #kernel_initializer=init
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=keras_model, epochs=10, batch_size=10, verbose=1)    #epochs=100
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
#optimizers = ['rmsprop', 'adam']
#init = ['glorot_uniform', 'normal', 'uniform']
#epochs = [2, 3, 4]              #epochs =[50, 100, 150]
#batches = [5, 10, 20]
param_grid = dict(neurons=neurons)              #optimizer=optimizers, epochs=epochs, batch_size=batches, init=init
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)    # cv=3
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
9111/9111 [==============================] - 7s 792us/step - loss: 1.9718 - accuracy: 0.4485
Epoch 2/10
9111/9111 [==============================] - 8s 840us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 3/10
9111/9111 [==============================] - 7s 786us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 4/10
9111/9111 [==============================] - 7s 810us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 5/10
9111/9111 [==============================] - 8s 836us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 6/10
9111/9111 [==============================] - 8s 897us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 7/10
9111/9111 [==============================] - ETA: 0s - loss: 1.9359 - accuracy: 0.44 - 8s 833us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 8/10
9111/9111 [==============================] - 7s 777us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 9/10
9111/9111 [==============================] - 7s 778us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 10/10
   1/4556 

4556/4556 [==============================] - 3s 645us/step - loss: 2.0069 - accuracy: 0.4281
Epoch 1/10
9111/9111 [==============================] - 7s 738us/step - loss: 2.0297 - accuracy: 0.4422
Epoch 2/10
9111/9111 [==============================] - 7s 741us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 3/10
9111/9111 [==============================] - 7s 735us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 4/10
9111/9111 [==============================] - 7s 742us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 5/10
9111/9111 [==============================] - 7s 730us/step - loss: 1.9474 - accuracy: 0.4437
Epoch 6/10
9111/9111 [==============================] - 7s 736us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 7/10
9111/9111 [==============================] - 7s 737us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 8/10
9111/9111 [==============================] - 7s 734us/step - loss: 1.9473 - accuracy: 0.4437
Epoch 9/10
9111/9111 [==============================] - 7s 735us/step - los

9111/9111 [==============================] - 8s 866us/step - loss: 1.7693 - accuracy: 0.4139
Epoch 2/10
9111/9111 [==============================] - 8s 853us/step - loss: 1.4219 - accuracy: 0.4507
Epoch 3/10
9111/9111 [==============================] - 8s 838us/step - loss: 1.3500 - accuracy: 0.4633
Epoch 4/10
9111/9111 [==============================] - 8s 848us/step - loss: 1.3149 - accuracy: 0.4692
Epoch 5/10
9111/9111 [==============================] - 8s 834us/step - loss: 1.2976 - accuracy: 0.4733
Epoch 6/10
9111/9111 [==============================] - 7s 819us/step - loss: 1.2818 - accuracy: 0.4763
Epoch 7/10
9111/9111 [==============================] - 8s 837us/step - loss: 1.2670 - accuracy: 0.4793
Epoch 8/10
9111/9111 [==============================] - 7s 814us/step - loss: 1.2550 - accuracy: 0.4834
Epoch 9/10
9111/9111 [==============================] - 7s 820us/step - loss: 1.2456 - accuracy: 0.4842
Epoch 10/10
4556/4556 [==============================] - 3s 654us/step - lo

### Second trial

In [4]:
%%time
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

# Define baseline model
def keras_model(neurons=1):                  #optimizer='rmsprop', init='glorot_uniform'
	# Create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=31, activation='relu'))        #kernel_initializer=init
	model.add(Dropout(0.2))                          
	model.add(Dense(11, activation='softmax'))        #kernel_initializer=init
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# Create model
model = KerasClassifier(build_fn=keras_model, epochs=10, batch_size=10, verbose=1)    #epochs=100
# Define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
#optimizers = ['rmsprop', 'adam']
#init = ['glorot_uniform', 'normal', 'uniform']
#epochs = [2, 3, 4]              #epochs =[50, 100, 150]
#batches = [5, 10, 20]
param_grid = dict(neurons=neurons)              #optimizer=optimizers, epochs=epochs, batch_size=batches, init=init
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)    # cv=3, (n_jobs=-1) <--- BAD IDEA
grid_result = grid.fit(X, Y)
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
9111/9111 [==============================] - 7s 766us/step - loss: 1.9707 - accuracy: 0.4486
Epoch 2/10
9111/9111 [==============================] - 7s 768us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 3/10
9111/9111 [==============================] - 7s 765us/step - loss: 1.9361 - accuracy: 0.4487
Epoch 4/10
9111/9111 [==============================] - 7s 764us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 5/10
9111/9111 [==============================] - 7s 766us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 6/10
9111/9111 [==============================] - 7s 780us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 7/10
9111/9111 [==============================] - 7s 807us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 8/10
9111/9111 [==============================] - 7s 803us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 9/10
9111/9111 [==============================] - 8s 876us/step - loss: 1.9362 - accuracy: 0.4487
Epoch 10/10
4556/4556 [==============================] - 3s 639u

In [ ]:
Sidenote ---> 8 ish different hyperparameter to tune